In [10]:
N_var = 3

In [11]:
import numpy as np
import pandas as pd

In [12]:
A_0 = np.identity(10) * 2 + np.eye(10, 10, k = -1) * -1 + np.eye(10, 10, k = 1) * -1

In [13]:
def construct_delta_A(N_var, eps):
    """
    Алгоритм получения матрицы delta_A
    Ввод: 
    N_var - номер варианта
    eps - параметр
    Вывод:
    delta_A
    """
    par = 10
    delta_A = []
    c = N_var * eps / (N_var + 1)
    for i in range(par):
        temp = []
        for j in range(par):
            temp.append(0 if i == j else c / (i + j))
        delta_A.append(temp)
    return np.array(delta_A)

In [14]:
eps = 0.1

In [15]:
def house(x):
    u_tilde = np.copy(x)
    u_tilde[0] += np.sign(x[0]) * np.linalg.norm(x)
    return u_tilde / np.linalg.norm(u_tilde)

def qr_householder(A):
    m, n = A.shape
    Q = np.identity(m)
    R = np.copy(A)
    for i in range(n):
        P_i         = np.identity(m) ## 1
        x           = R[i:, i] ## 2
        u           = house(x) ## 3
        P_streak    = np.identity(m - i) - 2 * np.outer(u, u) ## 4
        P_i[i:, i:] = np.copy(P_streak) ## 5
        R           = np.dot(P_i, R) ## 6
        Q           = Q @ P_i ## 7
    return Q[:, :n], R[:n, :] ## Обрезаем для матрицы Q (m-n) столбцов, для матрицы R - (m-n) строк

def qr_householder_effective(A):
    A_temp = np.copy(A)
    l = []
    m, n = A.shape
    for i in range(n):
        x                  = A_temp[i:, i] 
        u                  = house(x) 
        A_temp[i:m, i:n]  -= 2 * np.outer(u, np.matmul(u, A_temp[i:m, i:n]))
        A_temp[i + 1:m, i] = u[1:]
        l.append(u[0])
    return A_temp, l

### Решение задачи

In [16]:
delta_A = construct_delta_A(N_var, 0.1)
A = A_0 + delta_A

amnt_columns = 1 # Количество удаляемых столбцов
A_hat = np.delete(A, [9 - i for i in range(amnt_columns)], axis = 1).round(2)
x_0 = np.random.rand(10 - amnt_columns) # Выберем случайный вектор x_0
b = np.dot(A_hat, x_0)
b_effective = np.copy(b)

Q, R = qr_householder(A_hat)
A_effective, u_first_elements = qr_householder_effective(A_hat)

for i in range(10 - amnt_columns):
    u = np.hstack((u_first_elements[i],A_effective[i + 1:, i]))
    gamma = -2 * np.inner(u, b_effective[i:])
    b_effective[i:] += gamma * u

assert ((Q @ R - A_hat) < 1e-6).all(), "QR разложение неправильное"

x = np.dot(np.linalg.inv(R) @ Q.T, b) # Решаем линейную задачу наименьших квадратов
x_effective = np.dot(np.linalg.inv(np.triu(A_effective[:-amnt_columns])), b_effective[:-amnt_columns])

value_estimation = np.linalg.norm(x - x_0) / np.linalg.norm(x_0) # Относительная погрешность
result = (f"Решение было получено. \n"
          f"x_0                       = {[round(i, 4) for i in x_0]} \n"
          f"Найдённое решение x       = {[round(i, 4) for i in x]} \n"
          f"Эффективное решение x     = {[round(i, 4) for i in x_effective]} \n"
          f"Относительная погрешность = {value_estimation:.4e}")

### Результаты расчётов

In [17]:
print(result)

Решение было получено. 
x_0                       = [0.3077, 0.3567, 0.2158, 0.6656, 0.1888, 0.921, 0.1381, 0.2263, 0.5892] 
Найдённое решение x       = [0.3077, 0.3567, 0.2158, 0.6656, 0.1888, 0.921, 0.1381, 0.2263, 0.5892] 
Эффективное решение x     = [0.3077, 0.3567, 0.2158, 0.6656, 0.1888, 0.921, 0.1381, 0.2263, 0.5892] 
Относительная погрешность = 3.9321e-16


## Задание №2

Для матрицы $A$ найти все ее собственные значения ($\lambda_j, \quad j = 1,\ldots, 10$) и собственные вектора ($z_j$ , с 2-нормой равной 1) с помощью неявного QR-алгоритма со сдвигом для трех вариантов: $\varepsilon = 10^{-1}, 10^{-3}, 10^{-6}$. 

По итогам расчетов нужно сделать сводную таблицу, в которой указать следующие величины: $\left | \lambda_j - \lambda_j^{0}  \right |$ и $\left \| z_j - z_j^{0} \right \|$  для $j = 1,\ldots,10$ .

In [18]:
eps = [1e-1, 1e-3, 1e-6]